# How to win baseball games

### Introduction

Baseball is an American past time, it's full of legend and lore, instilled deep within the fabric of the American dream. To many it is more than a game. However as data scientists, we don't worry ourselves too much about the emotional aspects of things, especially when there is a quantitative advantage to be had. The name of the game is Baseball and the goal is to score runs in order to win. The question is; how do you score more runs? Luckily for us, Baseball is a sport in which statistics are meticulously recorded and scrutinized. In this notebook, we'll take the romantic aspects of the game and put them aside, we'll be using numbers and statistical analysis to build a winning team.

### The Data

The dataset we will be using Kaggle's 'The History of Baseball' which is a reformatted version of 'Lahman's Baseball Database'. It contains data from 1871 to 2015. The data is stored in the form of a small SQLite database.

In [1]:
# loading required modules 
import pandas as pd
import numpy as np
import sqlite3
conn = sqlite3.connect('/Users/robinphetsavong/Documents/robinphetsa.github.io/data_science/data-projects/Projects/Baseball/baseball/database.sqlite')
c = conn.cursor()

In [40]:
#Read in data using sql

batting_data = pd.read_sql_query("SELECT t.team_id,b.year,t.w as wins,t.l as losses,SUM(b.r) as runs,SUM(b.h) as hits,SUM(b.ab) as atbats,SUM(b.sf) as sacflies,SUM(b.hbp) as hitbypitch,SUM(b.bb) as baseonballs,SUM(b.double) as doubles,SUM(b.triple) as triples,SUM(b.hr) as hr,SUM(b.rbi) as rbi,SUM(b.sb) as sb,SUM(s.salary) as payroll FROM batting b INNER JOIN team t on b.team_id = t.team_id and b.year = t.year LEFT JOIN salary s on b.team_id = s.team_id and b.year = s.year and b.player_id = s.player_id GROUP BY t.team_id, b.year ORDER BY b.year, t.team_id;", conn)

In [41]:
batting_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2805 entries, 0 to 2804
Data columns (total 16 columns):
team_id        2805 non-null object
year           2805 non-null int64
wins           2805 non-null int64
losses         2805 non-null int64
runs           2805 non-null float64
hits           2805 non-null float64
atbats         2805 non-null float64
sacflies       2805 non-null float64
hitbypitch     2805 non-null float64
baseonballs    2805 non-null float64
doubles        2805 non-null float64
triples        2805 non-null float64
hr             2805 non-null float64
rbi            2805 non-null float64
sb             2805 non-null float64
payroll        888 non-null float64
dtypes: float64(12), int64(3), object(1)
memory usage: 350.7+ KB


### Feature Engineering

We'll be adding three more statistics to the dataframe:
* Batting Average
* On Base Percentage
* Slugging Percentage
* On Base and Slugging Percentage

#### Batting Average

In [25]:
# create a function that calculates the batting average
def get_bavg(x):
    H = x['hits']
    AB = x['atbats']
    if AB == 0 or H == 0:
        return 0
    else:
        return H/AB

In [42]:
# apply the function to our dataframe
batting_data['ba'] = batting_data.apply(get_bavg, axis=1)

##### On Base Percentage

In [27]:
# define a function to calculate the OBP that takes in a dataframe as a parameter
def get_obp(x):
    H = x['hits']
    BB = x['baseonballs']
    HBP = x['hitbypitch']
    SF = x['sacflies']
    AB = x['atbats']
    Denominator = (AB+BB+HBP+SF)
#    if Denominator == 0:
#        Denominator = 0.001 #prevent python from approximating to zero and getting div by zero errors
    if AB == 0:
        return 0
    else:
        return (H+BB+HBP)/Denominator

In [43]:
batting_data['obp'] = batting_data.apply(get_obp, axis=1)

##### Singles

In [29]:
# Function to calculate singles
def get_singles(x):
    H = x['hits']
    doubles = x['doubles']
    triples = x['triples']
    HR = x['hr']
    singles = H-(doubles-triples-HR)
    if singles <=0:
        return 0
    else:
        return singles

In [44]:
batting_data['singles'] = batting_data.apply(get_singles, axis=1)

##### Slugging Percentage

In [33]:
def get_slg(x):
    H = x['hits']
    doubles = x['doubles']
    triples = x['triples']
    HR = x['hr']
    singles = x['singles']
    AB = x['atbats']
    if AB == 0:
        return 0
    else:
        return (singles + 2*doubles + 3*triples + 4*HR)/(AB)

In [45]:
batting_data['slg'] = batting_data.apply(get_slg,axis=1)

#### On-base plus slugging

In [54]:
def get_ops(x):
    obp = x['obp']
    slg = x['slg']
    return obp+slg

In [55]:
batting_data['ops'] = batting_data.apply(get_ops,axis=1)

In [56]:
# Lets check our data

batting_data.head(7)

,team_id,year,wins,losses,runs,hits,atbats,sacflies,hitbypitch,baseonballs,...,triples,hr,rbi,sb,payroll,ba,obp,singles,slg,ops
0,BS1,1871,20,10,401,426,1372,0,0,60,...,37,3,267,73,NaN,0.310496,0.339385,396,0.480321,0.819706
1,CH1,1871,19,9,302,323,1196,0,0,60,...,21,10,193,69,NaN,0.270067,0.304936,302,0.425585,0.730522
2,CL1,1871,10,19,249,328,1186,0,0,26,...,40,7,164,18,NaN,0.276560,0.292079,340,0.470489,0.762568
3,FW1,1871,7,12,137,178,746,0,0,33,...,8,2,94,16,NaN,0.238606,0.270860,169,0.320375,0.591235
4,NY2,1871,16,17,302,403,1404,0,0,33,...,21,1,213,46,NaN,0.287037,0.303410,382,0.381054,0.684464
5,PH1,1871,21,7,376,410,1281,0,0,46,...,27,9,250,56,NaN,0.320062,0.343632,380,0.491023,0.834655
6,RC1,1871,4,21,231,274,1036,0,0,38,...,25,3,147,53,NaN,0.264479,0.290503,258,0.417954,0.708456


#### Cleaning up the dataframe

In [47]:
batting_data['runs'] = batting_data['runs'].astype(int)
batting_data['hits'] = batting_data['hits'].astype(int)
batting_data['atbats'] = batting_data['atbats'].astype(int)
batting_data['sacflies'] = batting_data['sacflies'].astype(int)
batting_data['hitbypitch'] = batting_data['hitbypitch'].astype(int)
batting_data['baseonballs'] = batting_data['baseonballs'].astype(int)
batting_data['doubles'] = batting_data['doubles'].astype(int)
batting_data['triples'] = batting_data['triples'].astype(int)
batting_data['hr'] = batting_data['hr'].astype(int)
batting_data['rbi'] = batting_data['rbi'].astype(int)
batting_data['sb'] = batting_data['sb'].astype(int)
batting_data['singles'] = batting_data['singles'].astype(int)

In [57]:
batting_data.tail(7)

,team_id,year,wins,losses,runs,hits,atbats,sacflies,hitbypitch,baseonballs,...,triples,hr,rbi,sb,payroll,ba,obp,singles,slg,ops
2798,SEA,2015,76,86,656,1379,5544,35,36,478,...,22,198,624,69,122208700.0,0.248737,0.310684,1337,0.490440,0.801125
2799,SFN,2015,84,78,696,1486,5565,37,49,457,...,39,136,663,93,164701500.0,0.267026,0.326130,1373,0.469003,0.795132
2800,SLN,2015,100,62,647,1386,5484,42,66,506,...,39,137,619,69,119241500.0,0.252735,0.321089,1274,0.458607,0.779696
2801,TBA,2015,80,82,644,1383,5485,47,84,436,...,32,167,612,87,62821233.0,0.252142,0.314442,1304,0.478396,0.792837
2802,TEX,2015,88,74,751,1419,5511,54,76,503,...,32,172,707,101,143742789.0,0.257485,0.325195,1344,0.487389,0.812584
2803,TOR,2015,93,69,891,1480,5509,62,54,570,...,17,232,852,88,112992400.0,0.268651,0.339629,1421,0.547468,0.887097
2804,WAS,2015,83,79,703,1363,5428,51,44,539,...,13,177,665,57,155587472.0,0.251105,0.321016,1288,0.472550,0.793566


In [58]:
# lets take a look at the columns we have now
batting_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2805 entries, 0 to 2804
Data columns (total 21 columns):
team_id        2805 non-null object
year           2805 non-null int64
wins           2805 non-null int64
losses         2805 non-null int64
runs           2805 non-null int64
hits           2805 non-null int64
atbats         2805 non-null int64
sacflies       2805 non-null int64
hitbypitch     2805 non-null int64
baseonballs    2805 non-null int64
doubles        2805 non-null int64
triples        2805 non-null int64
hr             2805 non-null int64
rbi            2805 non-null int64
sb             2805 non-null int64
payroll        888 non-null float64
ba             2805 non-null float64
obp            2805 non-null float64
singles        2805 non-null int64
slg            2805 non-null float64
ops            2805 non-null float64
dtypes: float64(5), int64(15), object(1)
memory usage: 460.3+ KB


In [65]:
import plotly.plotly as py
import plotly.graph_objs as go
from plotly import tools

series1 = go.Scatter(
    x = batting_data.obp,
    y = batting_data.runs,
    mode = 'markers',
    name= 'obp vs runs',
    marker= dict(size= 1,
                    line= dict(width=1),
                    color = '#ffff00'
                   ),
)


series2 = go.Scatter(
    x = batting_data.hr,
    y = batting_data.runs,
    mode = 'markers',
    name= 'hr vs runs',
    marker= dict(size= 2,
                    line= dict(width=1),
                    color = '#009933'
                   ),
)




data = [series1]

layout = dict(title = 'Correlation between Runs and OBP',
              xaxis = dict(title = 'On Base Percentage'),
              yaxis= dict(title = 'Runs Scored')
             )

fig = dict(data=data, layout=layout)

py.iplot(fig)